Going to acquire only data about left handed pitchers, going to add more columns so that the algorithm has more to process

In [1]:
#reading in original CSV file with all of the data
import pandas as pd
initial_data = pd.read_csv('./data/combined_data.csv')

In [2]:
#looking at the columns
initial_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'index', 'pitch_type', 'game_date',
       'release_speed', 'release_pos_x', 'release_pos_z', 'player_name',
       'batter', 'pitcher', 'events', 'description', 'spin_dir',
       'spin_rate_deprecated', 'break_angle_deprecated',
       'break_length_deprecated', 'zone', 'des', 'p_throws', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'pfx_x', 'pfx_z',
       'plate_x', 'plate_z', 'outs_when_up', 'inning', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'sv_id', 'vx0', 'vy0', 'vz0',
       'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed',
       'launch_angle', 'effective_speed', 'release_spin_rate',
       'release_extension', 'game_pk', 'pitcher.1', 'fielder_2.1',
       'release_pos_y', 'estimated_ba_using_speedangle',
       'estimated_woba_using_speedangle', 'woba_value', 'woba_denom',
       'babip_value', 'iso_value', 'launch_speed_angle', 'at_bat_number',
       'pitch_number', 'pitch_n

In [ ]:
#looking to add release extension, 
#the vs are all velocity per second in that direction
#the as are all acceleration

In [8]:
#break angle deprec is null
#break length is null
#ax and vx0 return values


In [2]:
#subsetting the dataframe for columns that we want to analyize
filter_data = initial_data[['pitch_name', 'release_speed', 'events', 'description', 'p_throws', 'type',\
                           'hit_distance_sc', 'launch_speed', 'launch_angle','launch_speed_angle', 'effective_speed', 'pitch_number', \
                           'spin_rate_deprecated', 'hit_location', 'bb_type', 'release_spin_rate', 'game_date' ,\
                         'pitch_number', 'vx0','vy0', 'vz0', 'ax', 'ay', 'az', 'pfx_x', 'pfx_z']]

In [3]:
#going to look at only left handed pitchers
lefty_filter = filter_data.loc[filter_data.p_throws == 'L']

In [6]:
#1.7 million rows
lefty_filter.shape

(1788537, 26)

In [4]:
#dropped irrelevant pitch names

lefty_filter = lefty_filter[(lefty_filter.pitch_name != 'Intentional Ball') 
            &(lefty_filter.pitch_name != 'Pitch Out') 
            & (lefty_filter.pitch_name != 'Fastball')
            &(lefty_filter.pitch_name != 'Unknown') ]

In [13]:
lefty_filter.shape

(1781396, 26)

In [5]:
#need to drop irrelevant descriptions that mislead to model to idetinfying an effective pitch
#need to combine swinging_strike_blocked with swinging strike

lefty_filter = lefty_filter[(lefty_filter.description != 'missed_bunt')
           & (lefty_filter.description != 'pitchout')
           & (lefty_filter.description != 'swinging_pitchout')
           & (lefty_filter.description != 'pitchout_hit_into_play')
           & (lefty_filter.description != 'pitchout_hit_into_play_score')
            & (lefty_filter.description != 'foul_pitchout')
            & (lefty_filter.description != 'unknown_strike')
            & (lefty_filter.description != 'pitchout_hit_into_play_no_out')
            ]


In [31]:
lefty_filter.shape

(1780079, 25)

In [6]:
#dropping irrelevant events
lefty_filter = lefty_filter[(lefty_filter.events != 'caught_stealing_2b')
           & (lefty_filter.events != 'pickoff_caught_stealing_2b')
            & (lefty_filter.events != 'pickoff_1b')
            & (lefty_filter.events != 'catcher_interf')
            & (lefty_filter.events != 'caught_stealing_3b')
            & (lefty_filter.events != 'pickoff_caught_stealing_3b')
            & (lefty_filter.events != 'pickoff_caught_stealing_home')
            & (lefty_filter.events != 'pickoff_3b')
            & (lefty_filter.events != 'batter_interference')
            & (lefty_filter.events != 'offensive_substitution')
            & (lefty_filter.events != 'umpire_substitution')
            & (lefty_filter.events != 'at_bat_start') 
            & (lefty_filter.events != 'run') 
            & (lefty_filter.events != 'pickoff_2b')
            & (lefty_filter.events != 'caught_stealing_home')
            ]

In [7]:
#1.7 million rows now
lefty_filter.shape

(1779102, 26)

In [18]:
#launch angle and spin rate not tracked in 2014
lefty_filter.loc[lefty_filter.game_date == '2014-08-23']

,pitch_name,release_speed,events,description,p_throws,type,hit_distance_sc,launch_speed,launch_angle,effective_speed,...,effective_speed,pitch_number,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z
3734004,4-Seam Fastball,87.6,single,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,...,NaN,1.0,-6.708,-127.442,-2.874,3.496,29.157,-23.813,0.169208,0.931000
3734130,Changeup,81.1,double,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,...,NaN,3.0,-4.256,-117.973,-3.093,8.397,23.382,-29.103,0.702217,0.506733
3734131,4-Seam Fastball,83.3,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,NaN,2.0,-4.224,-121.220,-0.493,1.694,24.875,-25.604,0.028650,0.829233
3734132,Curveball,74.3,NaN,called_strike,L,S,NaN,NaN,NaN,NaN,...,NaN,1.0,-0.886,-108.190,1.964,-4.915,20.529,-36.556,-0.713108,-0.343233
3734133,2-Seam Fastball,86.3,single,hit_into_play_score,L,X,NaN,NaN,NaN,NaN,...,NaN,3.0,-4.960,-125.544,-4.476,9.032,26.430,-23.300,0.660467,0.994067
3734134,Changeup,81.6,NaN,foul,L,S,NaN,NaN,NaN,NaN,...,NaN,2.0,-4.027,-118.718,-2.707,14.619,24.868,-25.421,1.310375,0.877967
3734135,2-Seam Fastball,86.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,NaN,1.0,-2.188,-125.486,-1.841,13.985,31.745,-23.210,1.123892,1.021300
3734136,2-Seam Fastball,86.3,walk,ball,L,B,NaN,NaN,NaN,NaN,...,NaN,4.0,-2.872,-125.663,-2.091,13.642,29.760,-23.746,1.079358,0.962533
3734137,Changeup,81.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,NaN,3.0,-1.924,-118.135,-0.495,10.070,26.855,-27.634,0.883133,0.662967
3734138,2-Seam Fastball,86.9,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,NaN,2.0,-3.015,-126.372,-5.013,11.111,31.201,-24.202,0.848342,0.916667


In [10]:
lefty_filter.pitch_name.value_counts()

4-Seam Fastball    600047
Slider             246045
2-Seam Fastball    240215
Changeup           226330
Sinker             188115
Curveball          149832
Cutter              83448
Knuckle Curve       29681
Split Finger         9510
Screwball             487
Eephus                337
Name: pitch_name, dtype: int64

In [7]:
#filtering out useless pitches

lefty_filter = lefty_filter[(lefty_filter.pitch_name != 'Screwball')
           & (lefty_filter.pitch_name != 'Eephus') ]

In [12]:
lefty_filter.pitch_name.value_counts()

4-Seam Fastball    600047
Slider             246045
2-Seam Fastball    240215
Changeup           226330
Sinker             188115
Curveball          149832
Cutter              83448
Knuckle Curve       29681
Split Finger         9510
Name: pitch_name, dtype: int64

In [66]:
lefty_filter.events.value_counts()

field_out                    187432
strikeout                     94522
single                        70129
walk                          35930
double                        20889
home_run                      12141
force_out                     10883
grounded_into_double_play      9714
field_error                    4379
hit_by_pitch                   3919
sac_bunt                       3320
sac_fly                        2975
triple                         1963
double_play                    1050
fielders_choice_out             796
strikeout_double_play           258
fielders_choice                 237
intent_walk                     125
other_out                        79
sac_fly_double_play              51
triple_play                      15
sac_bunt_double_play              3
Name: events, dtype: int64

In [13]:
#gonna change the description column
strike = ['called_strike','foul' ,'foul_tip','foul_bunt' ]
swinging_strike = ['swinging_strike','swinging_strike_blocked' ]
ball = ['intent_ball', 'blocked_ball']
def ChangeDescription(value):
    if value in strike:
        value = 'strike'
        return value
    elif value in swinging_strike:
        value = 'swinging_strike'
        return value
    elif value in ball:
        value = 'ball'
        return value
    else:
        return value

In [36]:
#changed description columns
lefty_filter.description = lefty_filter.description.apply(ChangeDescription)

In [37]:
lefty_filter.description.value_counts()

ball                    642969
strike                  633166
hit_into_play           211731
swinging_strike         172246
hit_into_play_no_out     75550
hit_into_play_score      38697
hit_by_pitch              3919
Name: description, dtype: int64

In [71]:
#looking at events column
lefty_filter.events.value_counts()

field_out                    187432
strikeout                     94522
single                        70129
walk                          35930
double                        20889
home_run                      12141
force_out                     10883
grounded_into_double_play      9714
field_error                    4379
hit_by_pitch                   3919
sac_bunt                       3320
sac_fly                        2975
triple                         1963
double_play                    1050
fielders_choice_out             796
strikeout_double_play           258
fielders_choice                 237
intent_walk                     125
other_out                        79
sac_fly_double_play              51
triple_play                      15
sac_bunt_double_play              3
Name: events, dtype: int64

In [8]:
out = ['field_out', 'force_out', 'sac_bunt', 'sac_fly', 'fielders_choice_out', 'fielders_choice', 'other_out']
double_play = ['grounded_into_double_play', 'double_play', 'strikeout_double_play', 'sac_fly_double_play', 'sac_bunt_double_play']
walk = ['walk', 'intent_walk']

def changeEvent(value):
    if value in out:
        value = 'out'
        return value
    elif value in double_play:
        value = 'double_play'
        return value
    elif value in walk:
        value = 'walk'
        return value
    else:
        return value

In [9]:
#for final project, going to simpliy double play to out
#going to change walk t0 ball
out = ['field_out', 'force_out', 'sac_bunt', 'sac_fly', 'fielders_choice_out', 'fielders_choice', 'other_out',\
        'grounded_into_double_play', 'double_play', 'strikeout_double_play', 'sac_fly_double_play', 'sac_bunt_double_play']
ball = ['walk', 'intent_walk']

def changeEvents(value):
    if value in out:
        value = 'out'
        return value
    elif value in ball:
        value = 'ball'
        return value
    else:
        return value

In [10]:
lefty_filter.events = lefty_filter.events.apply(changeEvents)

In [11]:
#events look good
lefty_filter.events.value_counts()

out             216798
strikeout        94522
single           70129
ball             36055
double           20889
home_run         12141
field_error       4379
hit_by_pitch      3919
triple            1963
triple_play         15
Name: events, dtype: int64

In [40]:
lefty_filter.columns

Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'pitch_number', 'spin_rate_deprecated',
       'hit_location', 'bb_type', 'release_spin_rate', 'game_date',
       'pitch_number', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'pfx_x',
       'pfx_z'],
      dtype='object')

In [41]:
#column not needed and not enough data
lefty_filter.drop(columns= ['hit_distance_sc'], inplace = True)

In [42]:
lefty_filter.effective_speed

60            NaN
61            NaN
62            NaN
63            NaN
64            NaN
65            NaN
66            NaN
67            NaN
68            NaN
69            NaN
70            NaN
71            NaN
72            NaN
328           NaN
329           NaN
330           NaN
331           NaN
332           NaN
333           NaN
334           NaN
335           NaN
336           NaN
337           NaN
338           NaN
339           NaN
340           NaN
341           NaN
342           NaN
343           NaN
348           NaN
            ...  
6486586    77.908
6486587    93.474
6486588    94.152
6486589    90.411
6486590    94.557
6486591    92.726
6486592    75.824
6486593    92.142
6486594    88.596
6486595    93.464
6486596    93.449
6486597    92.606
6486598    83.384
6486599    75.930
6486600    84.351
6486601    92.866
6486602    87.385
6486630    86.333
6486631    92.140
6486632    84.681
6486633    95.782
6486634    90.540
6486635    76.052
6486636    89.363
6486637   

In [43]:
lefty_filter.drop(columns=[ 'pitch_number', 'spin_rate_deprecated',], inplace = True)

In [44]:
lefty_filter.columns


Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'launch_speed', 'launch_angle', 'effective_speed',
       'hit_location', 'bb_type', 'release_spin_rate', 'game_date', 'vx0',
       'vy0', 'vz0', 'ax', 'ay', 'az', 'pfx_x', 'pfx_z'],
      dtype='object')

In [45]:
lefty_filter.drop(columns=['launch_speed','launch_angle', 'hit_location' ], inplace=True)

In [49]:
#still missing quite a few columns
lefty_filter.loc[lefty_filter.game_date == '2014-05-23']

,pitch_name,release_speed,events,description,p_throws,type,effective_speed,bb_type,release_spin_rate,game_date,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z
4087470,4-Seam Fastball,96.2,single,hit_into_play_no_out,L,X,NaN,fly_ball,NaN,2014-05-23,-8.247,-139.512,-9.213,14.454,35.243,-10.780,0.904008,1.770933
4087471,4-Seam Fastball,96.3,NaN,ball,L,B,NaN,NaN,NaN,2014-05-23,-7.337,-139.617,-9.682,8.846,36.385,-11.222,0.504600,1.742267
4087472,4-Seam Fastball,95.4,NaN,swinging_strike,L,S,NaN,NaN,NaN,2014-05-23,-5.362,-138.441,-9.298,11.982,36.019,-9.843,0.745358,1.874133
4087473,4-Seam Fastball,95.3,NaN,swinging_strike,L,S,NaN,NaN,NaN,2014-05-23,-5.436,-138.392,-8.559,16.401,33.119,-13.534,1.055700,1.583167
4087474,4-Seam Fastball,93.3,single,hit_into_play_no_out,L,X,NaN,line_drive,NaN,2014-05-23,-4.820,-135.867,-4.669,12.649,32.835,-15.403,0.821900,1.494300
4087475,4-Seam Fastball,93.8,NaN,strike,L,S,NaN,NaN,NaN,2014-05-23,-6.403,-136.178,-7.165,14.554,32.466,-14.746,0.958283,1.537300
4087476,4-Seam Fastball,94.8,NaN,ball,L,B,NaN,NaN,NaN,2014-05-23,-4.724,-137.616,-9.286,14.293,32.936,-13.666,0.916533,1.588900
4087477,2-Seam Fastball,94.1,NaN,ball,L,B,NaN,NaN,NaN,2014-05-23,-6.689,-136.491,-10.172,14.760,31.222,-15.887,0.963850,1.436967
4087478,4-Seam Fastball,94.3,walk,ball,L,B,NaN,NaN,NaN,2014-05-23,-6.040,-137.142,-2.810,13.127,33.121,-14.070,0.838600,1.570267
4087479,2-Seam Fastball,94.3,NaN,strike,L,S,NaN,NaN,NaN,2014-05-23,-3.843,-137.039,-7.669,15.385,30.879,-16.377,0.998642,1.388233


In [12]:
statcast_data = lefty_filter.loc[lefty_filter.game_date >= '2015']

In [51]:
statcast_data.columns

Index(['pitch_name', 'release_speed', 'events', 'description', 'p_throws',
       'type', 'effective_speed', 'bb_type', 'release_spin_rate', 'game_date',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'pfx_x', 'pfx_z'],
      dtype='object')

In [20]:
statcast_data.events.value_counts()

out             91605
strikeout       43285
single          29960
ball            15892
double           8831
home_run         5753
field_error      1889
hit_by_pitch     1817
triple            887
triple_play         8
Name: events, dtype: int64

In [24]:
statcast_data

,pitch_name,release_speed,events,description,p_throws,type,hit_distance_sc,launch_speed,launch_angle,launch_speed_angle,...,pitch_number,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z,bauer_units
2874428,Cutter,90.4,out,line_drive,L,X,256.0,79.1,22.0,4.0,...,6.0,-7.0650,-131.5120,-3.2980,-3.3340,27.8610,-30.5200,-0.394417,0.329000,27.090708
2874429,Slider,81.5,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,5.0,-7.9400,-118.3900,-2.9300,-0.9770,18.7140,-31.7650,-0.224633,0.232967,28.134969
2874430,Curveball,77.1,NaN,blocked_ball,L,B,NaN,NaN,NaN,NaN,...,4.0,-6.2900,-112.1490,-2.4420,-4.2520,25.1530,-42.5170,-0.607342,-1.004000,32.983139
2874431,Cutter,87.6,NaN,called_strike,L,S,NaN,NaN,NaN,NaN,...,3.0,-5.7420,-127.4870,-2.0310,0.4720,27.3210,-28.6500,-0.089642,0.502433,28.687215
2874432,Cutter,86.2,NaN,called_strike,L,S,NaN,NaN,NaN,NaN,...,2.0,-6.8740,-125.3770,-2.6520,-1.3590,24.3800,-28.3090,-0.248292,0.539700,28.097448
2874433,4-Seam Fastball,91.8,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,1.0,-9.2520,-133.4200,-2.6920,10.4440,31.3750,-19.8110,0.684125,1.187567,23.039216
2874434,Cutter,88.1,single,ground_ball,L,X,60.0,89.1,3.0,2.0,...,4.0,-5.2720,-128.2350,-2.3900,-1.4730,28.0230,-28.6660,-0.253858,0.498133,27.979569
2874435,Curveball,77.2,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,3.0,-4.6200,-112.3640,-0.7270,-5.7530,24.9120,-42.3200,-0.771558,-0.973900,32.564767
2874436,Slider,80.0,NaN,ball,L,B,NaN,NaN,NaN,NaN,...,2.0,-1.0720,-116.5000,-0.5600,-1.2300,22.3110,-35.8290,-0.255250,-0.189867,NaN
2874437,4-Seam Fastball,89.5,NaN,foul,L,S,233.0,75.0,35.0,NaN,...,1.0,-7.9270,-130.1040,-3.6470,3.1830,27.4530,-22.1590,0.128850,1.031333,27.240223


In [99]:
statcast_data.bb_type.value_counts()

ground_ball    64707
line_drive     35006
fly_ball       29405
popup           9659
Name: bb_type, dtype: int64

In [103]:
statcast_data.description.value_counts()

ball                    280429
strike                  275177
hit_into_play            89517
swinging_strike          80303
hit_into_play_no_out     32247
hit_into_play_score      17013
hit_by_pitch              1817
Name: description, dtype: int64

In [ ]:
#for launch speed, launch angle, and hit location need to make NaN values = 0
#could also create bauer units
#see if other metric
#if i combine bb_type and description that will do away with batted ball type

In [13]:
statcast_data['bauer_units'] = statcast_data['release_spin_rate']/ statcast_data['release_speed']

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
#renaming description based on hit into play
import numpy as np
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play', statcast_data['bb_type'],\
                                     statcast_data['description'])
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play_no_out', statcast_data['bb_type'],\
                                     statcast_data['description'])
statcast_data['description'] = np.where(statcast_data['description'] == 'hit_into_play_score', statcast_data['bb_type'],\
                                     statcast_data['description'])

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [23]:
statcast_data.description.value_counts()

ball                       261567
foul                       134461
called_strike              131932
swinging_strike             73736
ground_ball                 64707
line_drive                  35006
fly_ball                    29405
blocked_ball                18783
popup                        9659
swinging_strike_blocked      6567
foul_tip                     6374
foul_bunt                    2410
hit_by_pitch                 1817
intent_ball                    79
Name: description, dtype: int64

In [14]:
import numpy as np
statcast_data.events = np.where(statcast_data.events.isna(), statcast_data.description, statcast_data.events)

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
statcast_data.events.sum()

In [15]:
#want to change events so that only possible events are: strike, ball, HBP, out,foul,  single. double, triple, HR
strike = ['called_strike', 'swinging_strike', 'strikeout', 'swinging_strike_blocked']
ball = ['ball', 'blocked_ball', 'intent_ball']
foul = ['foul', 'foul_tip', 'foul_bunt']
out = ['triple_play']
def eventchange(value):
    if value in strike:
        value = 'strike'
        return value
    elif value in ball:
        value = 'ball'
        return value
    elif value in foul:
        value = 'foul'
        return value
    elif value in out:
        value = 'out'
        return value
    else:
        return value

In [16]:
statcast_data.events = statcast_data.events.apply(eventchange)

In [17]:
statcast_data

,pitch_name,release_speed,events,description,p_throws,type,hit_distance_sc,launch_speed,launch_angle,launch_speed_angle,...,pitch_number,vx0,vy0,vz0,ax,ay,az,pfx_x,pfx_z,bauer_units
2874428,Cutter,90.4,out,hit_into_play,L,X,256.0,79.1,22.0,4.0,...,6.0,-7.0650,-131.5120,-3.2980,-3.3340,27.8610,-30.5200,-0.394417,0.329000,27.090708
2874429,Slider,81.5,ball,ball,L,B,NaN,NaN,NaN,NaN,...,5.0,-7.9400,-118.3900,-2.9300,-0.9770,18.7140,-31.7650,-0.224633,0.232967,28.134969
2874430,Curveball,77.1,ball,blocked_ball,L,B,NaN,NaN,NaN,NaN,...,4.0,-6.2900,-112.1490,-2.4420,-4.2520,25.1530,-42.5170,-0.607342,-1.004000,32.983139
2874431,Cutter,87.6,strike,called_strike,L,S,NaN,NaN,NaN,NaN,...,3.0,-5.7420,-127.4870,-2.0310,0.4720,27.3210,-28.6500,-0.089642,0.502433,28.687215
2874432,Cutter,86.2,strike,called_strike,L,S,NaN,NaN,NaN,NaN,...,2.0,-6.8740,-125.3770,-2.6520,-1.3590,24.3800,-28.3090,-0.248292,0.539700,28.097448
2874433,4-Seam Fastball,91.8,ball,ball,L,B,NaN,NaN,NaN,NaN,...,1.0,-9.2520,-133.4200,-2.6920,10.4440,31.3750,-19.8110,0.684125,1.187567,23.039216
2874434,Cutter,88.1,single,hit_into_play_no_out,L,X,60.0,89.1,3.0,2.0,...,4.0,-5.2720,-128.2350,-2.3900,-1.4730,28.0230,-28.6660,-0.253858,0.498133,27.979569
2874435,Curveball,77.2,ball,ball,L,B,NaN,NaN,NaN,NaN,...,3.0,-4.6200,-112.3640,-0.7270,-5.7530,24.9120,-42.3200,-0.771558,-0.973900,32.564767
2874436,Slider,80.0,ball,ball,L,B,NaN,NaN,NaN,NaN,...,2.0,-1.0720,-116.5000,-0.5600,-1.2300,22.3110,-35.8290,-0.255250,-0.189867,NaN
2874437,4-Seam Fastball,89.5,foul,foul,L,S,233.0,75.0,35.0,NaN,...,1.0,-7.9270,-130.1040,-3.6470,3.1830,27.4530,-22.1590,0.128850,1.031333,27.240223


In [109]:
#still need to do something with NaN values, 
#liking this dataset a lot more though
statcast_data.to_csv('statcast_data.csv')

In [18]:
statcast_data.to_csv('predict_outcome.csv')

Tuesday 3/12
Working on filling in NaN values. Dropping irrelevant columns. Possibly one more calculated column
Feed to few models to determine best fit

In [1]:
import pandas as pd
lefty_data = pd.read_csv('statcast_data.csv')

In [56]:
#revising so still working with statcast data
statcast_data.drop(columns=[ 'p_throws', 'bb_type'], inplace= True)

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [2]:
#drop columns unnamed, p_throws, game_date, bb_Type since that is now reflected in description
lefty_data.drop(columns=['Unnamed: 0', 'p_throws', 'bb_type', 'game_date'], inplace= True)

In [60]:
statcast_data.isna().sum()

pitch_name            2082
release_speed         1768
events                   0
description              0
type                     0
effective_speed       9627
release_spin_rate    29078
game_date                0
vx0                   1835
vy0                   1835
vz0                   1835
ax                    1835
ay                    1835
az                    1835
pfx_x                 1835
pfx_z                 1835
bauer_units          29078
dtype: int64

In [57]:
lefty_data.isnull().sum()

NameError: name 'lefty_data' is not defined

In [14]:
lefty_data['events'].value_counts()

out             86790
strikeout       43285
single          29960
walk            15892
double           8831
home_run         5753
double_play      4815
field_error      1889
hit_by_pitch     1817
triple            887
triple_play         8
Name: events, dtype: int64

In [4]:
import numpy as np
lefty_data.pitch_name.isna().value_counts()

False    774421
True       2082
Name: pitch_name, dtype: int64

In [5]:
#instead of Nan going to put a zero for hit location
lefty_data.hit_location = np.where(lefty_data.hit_location.isnull(), 0, lefty_data.hit_location)

In [6]:
#changed launch speed to zero for balls that were not hit
lefty_data.launch_speed = np.where(lefty_data.launch_speed.isnull(), 0, lefty_data.launch_speed)

In [7]:
#might look to have launch angle be mean
lefty_data.launch_angle.mean()

14.462875312862165

In [8]:
#changed average launch angle to 14
lefty_data.launch_angle = np.where(lefty_data.launch_angle.isna(), lefty_data.launch_angle.mean(), lefty_data.launch_angle)

In [10]:
#going to replace events with an X to indicate not applicable 
lefty_data.events = np.where(lefty_data.events.isna(), 'X', lefty_data.events)

In [59]:
#for revised data
statcast_data.events = np.where(statcast_data.events.isna(), 'X', statcast_data.events)

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [12]:
#going to drop type column since it is reflected in description column
lefty_data.drop(columns=['type'], inplace = True)

In [35]:
condition = lefty_data[((lefty_data.description == 'swing_strike') & (lefty_data.events == 'strikeout'))]

In [36]:
#lots of swinging strikes are swingstrike outs so going to reflect that in the description column
lefty_data.description = np.where(condition, 'swinging_strikeout', lefty_data.description)

ValueError: operands could not be broadcast together with shapes (0,9) () (776503,) 

In [49]:
lefty_data.isna().sum()

pitch_name            2082
release_speed         1768
events                   0
description              0
launch_speed             0
launch_angle             0
hit_location             0
release_spin_rate    29078
bauer_units          29078
dtype: int64

In [66]:
#dropping nan values from pitch names
lefty_data.pitch_name.dropna(inplace=True)

In [50]:
lefty_data.pitch_name.isna().value_counts()

False    774421
Name: pitch_name, dtype: int64

In [68]:
lefty_data.release_speed.isna().value_counts()

False    774735
True       1768
Name: release_speed, dtype: int64

In [69]:
#dropping nan values from release speed
lefty_data.release_speed.dropna(inplace=True)

In [70]:
lefty_data.release_speed.isna().value_counts()

False    774735
Name: release_speed, dtype: int64

In [56]:
lefty_data.release_spin_rate.mean()

2157.6666180553234

In [60]:
lefty_data.bauer_units.mean()

24.636406664076294

In [61]:
#changing null values to mean spin rate
lefty_data.release_spin_rate = np.where(lefty_data.release_spin_rate.isna(),lefty_data.release_spin_rate.mean(),\
                                      lefty_data.release_spin_rate )

In [63]:
#changing null values to mean bauer units
lefty_data.bauer_units = np.where(lefty_data.bauer_units.isna(), lefty_data.bauer_units.mean(),\
                                 lefty_data.bauer_units)

In [80]:
lefty_data.hit_location.isna().value_counts()

False    776503
Name: hit_location, dtype: int64

In [81]:
lefty_data.to_csv('model_lefty.csv')

In [63]:
#revised data set with more pitch columns
statcast_data.dropna(inplace=True)

/Users/nickrad/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [65]:
statcast_data.to_csv('lefty_revised.csv')